<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2025/blob/main/BandBroadening_StirredTanksInSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig

In [21]:
N = 50
c_ss = np.full(N, 1.)

totV = 1. #L
V = totV/N
q = 0.1 #L/s
tend= 50. #s

In [22]:
def rhs(t,c):
    dc=np.zeros_like(c)
    cin = 3
    dc[0] = q*(cin - c[0])/V
    dc[1:] = q*(c[:-1]-c[1:])/V
    return dc

In [23]:
res=sp.integrate.solve_ivp(rhs, (0,tend), c_ss, method='Radau', dense_output=True)

In [24]:
tplot = np.linspace(0,tend,100)
cs = res.sol(tplot)

In [25]:
fig = make_subplots()
for i, c in enumerate(cs):
    fig.add_scatter(x=tplot,y=c,mode='lines', name=f'Tank{i+1}')
fig.show()